# Upgrading from MathProgBase.jl to MathOptInterface.jl

+ MathProgBase.jl has been [depreciated](https://github.com/JuliaOpt/MathProgBase.jl/pull/231) since Jan 2019. It is therefore not officially supported by julia v1.2 and above. We should use [MathOptInterface.jl](https://github.com/jump-dev/MathOptInterface.jl) instead. 
+ Other packages such as BinaryProvider.jl and Ipopt.jl needs to also be updated to the latest version or depreciated and use the latest one supported by the Julia community.
+ To ensure a smooth update, checkout the [official documentation](https://jump.dev/MathOptInterface.jl/dev/tutorials/mathprogbase/) as well as examples by [WiSER.jl](https://github.com/OpenMendel/WiSER.jl/blob/master/src/fit.jl) and [OrdinalMultinomialModels.jl](https://github.com/OpenMendel/OrdinalMultinomialModels.jl/blob/master/src/ordmnfit.jl)

In [1]:
using Revise
using Ipopt
using QuasiCopula
using Random
using GLM
using LinearAlgebra
using Statistics
using StatsBase
using MathOptInterface
using RCall

ENV["COLUMNS"] = 240
BLAS.set_num_threads(1)
Threads.nthreads()

Precompiling QuasiCopula
        Info Given QuasiCopula was explicitly requested, output will be shown live 
ERROR: LoadError: UndefVarError: `eval_f` not defined
Stacktrace:
 [1] getproperty(x::Module, f::Symbol)
   @ Base ./Base.jl:31
 [2] top-level scope
   @ ~/.julia/dev/QuasiCopula/src/parameter_estimation/fit_glm_ar_cs.jl:166
 [3] include(mod::Module, _path::String)
   @ Base ./Base.jl:495
 [4] include(x::String)
   @ QuasiCopula ~/.julia/dev/QuasiCopula/src/QuasiCopula.jl:1
 [5] top-level scope
   @ ~/.julia/dev/QuasiCopula/src/QuasiCopula.jl:55
 [6] include
   @ ./Base.jl:495 [inlined]
 [7] include_package_for_output(pkg::Base.PkgId, input::String, depot_path::Vector{String}, dl_load_path::Vector{String}, load_path::Vector{String}, concrete_deps::Vector{Pair{Base.PkgId, UInt128}}, source::Nothing)
   @ Base ./loading.jl:2222
 [8] top-level scope
   @ stdin:3
in expression starting at /Users/biona001/.julia/dev/QuasiCopula/src/parameter_estimation/fit_glm_ar_cs.jl:166
in express

LoadError: The following 1 direct dependency failed to precompile:

QuasiCopula [c47b6ae2-b804-4668-9957-eb588c99ffbc]

Failed to precompile QuasiCopula [c47b6ae2-b804-4668-9957-eb588c99ffbc] to "/Users/biona001/.julia/compiled/v1.10/QuasiCopula/jl_oacKF7".
[91m[1mERROR: [22m[39mLoadError: UndefVarError: `eval_f` not defined
Stacktrace:
 [1] [0m[1mgetproperty[22m[0m[1m([22m[90mx[39m::[0mModule, [90mf[39m::[0mSymbol[0m[1m)[22m
[90m   @[39m [90mBase[39m [90m./[39m[90m[4mBase.jl:31[24m[39m
 [2] top-level scope
[90m   @[39m [90m~/.julia/dev/QuasiCopula/src/parameter_estimation/[39m[90m[4mfit_glm_ar_cs.jl:166[24m[39m
 [3] [0m[1minclude[22m[0m[1m([22m[90mmod[39m::[0mModule, [90m_path[39m::[0mString[0m[1m)[22m
[90m   @[39m [90mBase[39m [90m./[39m[90m[4mBase.jl:495[24m[39m
 [4] [0m[1minclude[22m[0m[1m([22m[90mx[39m::[0mString[0m[1m)[22m
[90m   @[39m [35mQuasiCopula[39m [90m~/.julia/dev/QuasiCopula/src/[39m[90m[4m

In [3]:
k = 0 # number of causal SNPs

qc_model, G, βtrue, θtrue, γtrue, τtrue = simulate_longitudinal_traits(
    n = 10000, # sample size
    d_min = 5, # min number of observations per sample
    d_max = 5, # max number of observations per sample
    p = 15, # number of fixed effects, including intercept
    m = 2, # number of variance components
    q = 1000, # number of SNPs
    k = k, # number of causal SNPs
    seed = 2024,
    y_distribution = Normal,
    τtrue = 0.5,
    T = Float64,
    maf = 0.3,
    causal_snp_β = 0.2
)

@show qc_model;

@time optm = QuasiCopula.fit!(qc_model);

qc_model = Quasi-Copula Variance Component Model
  * base distribution: Normal
  * link function: IdentityLink
  * number of clusters: 10000
  * cluster size min, max: 5, 5
  * number of variance components: 2
  * number of fixed effects: 15


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.14, running with linear solver MUMPS 5.6.2.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:       18
                     variables with only 

In [4]:
@show βtrue
@show qc_model.β
@show qc_model.∇β

@show θtrue
@show qc_model.θ
@show qc_model.∇θ;

@show τtrue
@show qc_model.τ
@show qc_model.∇τ;

βtrue = [1.0, -0.5, 0.5, 0.5, 0.5, -0.5, 0.5, -0.5, 0.5, -0.5, -0.5, -0.5, 0.5, -0.5, -0.5]
qc_model.β = [1.0015558069145196, -0.4992552816604949, 0.5003258775344148, 0.5104867759085501, 0.503433481674981, -0.5109228927293467, 0.48934968009349356, -0.5029571938424525, 0.5049029761584054, -0.5083950994921372, -0.4838377755959444, -0.5082673883570834, 0.508559185132388, -0.49494971764139967, -0.5024689132271957]
qc_model.∇β = [-0.0004174672775286853, 1.855972752784485e-5, 0.00013656486524218914, 9.337930918396076e-6, 1.9344740755378176e-5, 2.752102204828244e-5, 3.267097026424248e-5, -0.00013902709178342798, -7.60145868601847e-5, 3.8777999550543285e-5, -4.696202700368857e-5, 2.4041912385336373e-5, -0.00010361037352457636, 5.2108730824396154e-5, 0.00010436649891751304]
θtrue = [0.1, 0.1]
qc_model.θ = [0.10841706495069867, 0.11742437271630245]
qc_model.∇θ = [3.458515023302411e-5, 3.302592342491195e-5]
τtrue = 0.5
qc_model.τ = [0.5080458900474164]
qc_model.∇τ = [0.0002994001751677189]


# AR

In [1]:
using Revise
using Ipopt
using QuasiCopula
using Random
using GLM
using LinearAlgebra
using Statistics
using StatsBase
using MathOptInterface
using RCall

Precompiling QuasiCopula
  ✓ QuasiCopula
  1 dependency successfully precompiled in 4 seconds. 151 already precompiled.


In [2]:
# get data
R"""
library("gcmr")
data("epilepsy", package = "gcmr")
"""
@rget epilepsy

# form model
df = epilepsy
y = :counts
grouping = :id
d = Normal() # Poisson()
link = IdentityLink() # LogLink()
covariates = [:visit, :trt]
qc_model = AR_model(df, y, grouping, covariates, d, link; penalized=true)

# fit
QuasiCopula.fit!(qc_model)

gcm.β = [31.41573849878938, -22.96610169491528, -0.339631336405546]
initializing dispersion using residual sum of squares
gcm.τ = [0.0037930980963653453]
initializing AR(1) noise paramter using MM-algorithm

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.14, running with linear solver MUMPS 5.6.2.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        6
                     variables with only lower bounds:        2
             

-1230.9645317304487

In [6]:
@show qc_model.β
@show qc_model.σ2
@show qc_model.τ
@show qc_model.ρ;

qc_model.β = [33.906723110174724, -22.966101624887735, -0.03905523987816367]
qc_model.σ2 = [0.2833334216154253]
qc_model.τ = [0.004425825590418177]
qc_model.ρ = [1.0000000099997475]


# CS

In [1]:
using Revise
using Ipopt
using QuasiCopula
using Random
using GLM
using LinearAlgebra
using Statistics
using StatsBase
using MathOptInterface
using RCall

Precompiling QuasiCopula
  ✓ QuasiCopula
  1 dependency successfully precompiled in 4 seconds. 151 already precompiled.


In [5]:
# get data
R"""
data(respiratory, package="geepack")
respiratory_df <- respiratory[order(respiratory$id),]
"""
@rget respiratory_df

# form model
df = respiratory_df
y = :outcome
grouping = :id
d = Bernoulli()
link = LogitLink()
covariates = [:center, :age, :baseline]
qc_model = CS_model(df, y, grouping, covariates, d, link)

# fit
QuasiCopula.fit!(qc_model)

initializing β using Newton's Algorithm under Independence Assumption

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.14, running with linear solver MUMPS 5.6.2.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        6
                     variables with only lower bounds:        1
                variables with lower and upper bounds:        1
                     variables with only upper bounds:        0
Total number of equali

-249.00152024184712

In [6]:
@show qc_model.β
@show qc_model.σ2
@show qc_model.ρ;

qc_model.β = [-0.807352363702958, 0.8553551273442161, -0.027821938178065966, 2.0702700293048473]
qc_model.σ2 = [0.3524250465323322]
qc_model.ρ = [0.8734708179288094]
